## System 3: Clean/Prepare Flight Operations Data

This system takes in operations data from Aerobahn and make it usable for the purposes of 400Hz analysis.

Most significantly, data from Aerobahn has multiple rows for each flight, one for the arrival and another for the departure. Since we need to know when a given plane was at a certain gate, we have to merge these rows into one that has columns for when the plane reached and left the gate. These values are derived from the "In Block" (for Operation = Arrival) and "Off Block" (for Operation = Departure) times.

**Inputs:**

- **3a:** (CSV) Operations data input from. Expected forms:
  - Each flight will have a separate row for 
  - Expected Columns:
    - `Carrier Group`
    - `Operation`
    - `Registration`
    - `Gate`
    - `Gate Assigned (Aerobahn)`
    - `International or Domestic Indicator`
    - `Model`
    - `Actual In Block Time (Aerobahn)`
    - `Actual Off Block Time (Aerobahn)`

  - The following columns are expected & preserved, but not used:
    - `Event Time`
    - `Call Sign`
    - `Destination Airport`
    - `Origin Airport`
    - `Actual Take Off Time (Aerobahn)`
    - `Actual Landing Time (Aerobahn)`  
    - `Scheduled In Block Time (Aerobahn)`
    - `Scheduled Off Block Time (Aerobahn)`
    - `Total Taxi Time`   
  - Example (not all columns shown): ![](screenshots/3a_sample_ops_data_input.png)
  
- **3b:** (Optional) Mapping from one gate name scheme to another (e.g. to convert SFO gate naming to original scheme, or the other way around) ![](screenshots/3b_sample_mapping.png)

**Output:**

* **3c:** Cleaned up operations data, with cleaned up data, per the following
  - Arrival and Departure rows are grouped if they have the same `Registration` and `Gate`  and are closest to each other and are within 12 hours of each other
  - `calculated_gate` = `Gate`, or, if not present, `Gate Assigned (Aerobahn)`
  - `call_sign_arrival` = `Call Sign` from the `Operation` = `Arrival` (arrival) row
  - `call_sign_departure` = `Call Sign` from the `Operation` = `Departure` (departure) row
  - `in_block_time_dt` = `Actual In Block Time (Aerobahn)` from the arrival row
  - `off_block_time_dt` = `Actual Off Block Time (Aerobahn)` from the departure row
  - `block_time_delta` = Difference between the two values above

In [1]:
# Dependencies

# For any missing libraries, just run (remove curly braces):
# !pip install {library_name}
 
## Required
import pandas as pd
import datetime as dt
from tqdm import tqdm_notebook as tqdm

In [2]:
# Inputs, settings, and toggles

# Location of Aerobahn input data
input_3a_location = 'sample_data/3a_sample_aerobahn_ops_data.csv'

# (Optional) Location of mapping for operations data from new to old gate naming scheme
# Leave it as an empty string if not necessary
# This was implemented because psat analysis & reports used the old scheme, as did the gate metering system
# Aerobahn, however, changed to the new gate naming system on 10/16/2019 at 00:00:00
# input_3b_location = '' # Keep as empty string if unnecessary
input_3b_location = 'sample_data/1b_mapping.csv'

# Filter on maximum amount of time between plane's arrival and departure from gate (default = 12 hours)
MAX_BLOCK_DIFFERENCE_TIME = pd.Timedelta('12 hours')

# Columns from input data to drop
columns_to_drop = ['Runway Assigned (Aerobahn)', 'Runway', 'Movement Area Exit Time']

# Location of output data
output_3c_location = 'sample_data/3c_sample_cleaned_operations_data.csv'

### Code begins below:

In [3]:
# Load in the data

ops_data = pd.read_csv(input_3a_location, low_memory=False)
ops_data.head()

,Carrier Group,Call Sign,Registration,Model,Operation,Origination Airport,Destination Airport,Gate Assigned (Aerobahn),Gate,Runway Assigned (Aerobahn),...,Event Time,Scheduled Off Block Time (Aerobahn),Actual Off Block Time (Aerobahn),Movement Area Entrance Time,Actual Take Off Time (Aerobahn),Actual Landing Time (Aerobahn),Movement Area Exit Time,Scheduled In Block Time (Aerobahn),Actual In Block Time (Aerobahn),Total Taxi Time
0,United Airlines,UAL257,N769UA,B772,Arrival,NaN,NaN,NaN,NaN,28R,...,9/12/19 0:00,NaN,NaN,NaN,NaN,9/12/19 0:00,NaN,NaN,NaN,NaN
1,United Airlines,UAL385,N57439,37K,Arrival,IAH,SFO,Gate_F73,Gate_F73,28R,...,9/12/19 0:02,9/11/19 16:31,9/11/19 20:10,NaN,9/11/19 20:27,9/12/19 0:02,9/12/19 0:05,9/11/19 20:44,9/12/19 0:06,0:04:25
2,Alaska Airlines,SKW3395,N404SY,E75,Arrival,PSP,SFO,HardStand_2_A13_A13B,HardStand_2_A13_A13B,28R,...,9/12/19 0:04,9/11/19 20:40,9/11/19 22:32,NaN,9/11/19 22:57,9/12/19 0:04,9/12/19 0:08,9/11/19 22:10,9/12/19 0:13,0:09:16
3,Delta Air Lines,DAL2416,N115DU,BCS1,Arrival,SLC,SFO,Gate_C42,Gate_C42,28R,...,9/12/19 0:05,9/11/19 21:25,9/11/19 22:00,NaN,9/11/19 22:31,9/12/19 0:05,9/12/19 0:08,9/11/19 23:30,9/12/19 0:08,0:03:25
4,United Airlines,SKW5557,N788SK,CR7,Arrival,SBA,SFO,Gate_F84D,Gate_F84D,28R,...,9/12/19 0:07,9/11/19 17:18,9/11/19 23:09,NaN,9/11/19 23:24,9/12/19 0:07,9/12/19 0:09,9/11/19 18:30,9/12/19 0:12,0:05:08


In [4]:
# Let's clean up our dataset to get rid of invalid values. 
# We want to see how much data is being lost at each step, so we'll print verbosely.

ops_data_cleaned = ops_data.copy()
print("Contains %d values to start with" % ops_data_cleaned.shape[0])

print("\nDeleting rows with no gate data...")
ops_data_cleaned = ops_data_cleaned.dropna(subset=['Gate Assigned (Aerobahn)', 'Gate'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

print("\nDeleting rows with no registration data...")
ops_data_cleaned = ops_data_cleaned.dropna(subset=['Registration'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

print("\nDeleting rows with no actual block in time or block out time (one or the other okay)...")
ops_data_cleaned = ops_data_cleaned.dropna(subset=['Actual Off Block Time (Aerobahn)', 'Actual In Block Time (Aerobahn)'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

print("\nDeleting columns we don't care for: %s" % str(columns_to_drop))
# Ignore errors in case these columns don't exist in provided data:
ops_data_cleaned = ops_data_cleaned.drop(columns=columns_to_drop, errors='ignore')
print("Contains %d values now" % ops_data_cleaned.shape[0])

print("\nDeleting rows with operations of Missed Approach or Rejected Take-off")
ops_data_cleaned = ops_data_cleaned[ops_data_cleaned['Operation'] != 'Missed Approach']
ops_data_cleaned = ops_data_cleaned[ops_data_cleaned['Operation'] != 'Rejected Take-off']
print("Contains %d values now" % ops_data_cleaned.shape[0])

Contains 170688 values to start with

Deleting rows with no gate data...
Contains 150022 values now

Deleting rows with no registration data...
Contains 150021 values now

Deleting rows with no actual block in time or block out time (one or the other okay)...
Contains 149782 values now

Deleting columns we don't care for: ['Runway Assigned (Aerobahn)', 'Runway', 'Movement Area Exit Time']
Contains 149782 values now

Deleting rows with operations of Missed Approach or Rejected Take-off
Contains 149255 values now


In [5]:
# Like we mentioned, the values we care most about are block-in and block-out times. We need to 
# ensure we have the correct values given the operation, so we're making our own block_time
# column with clean data.
print("Making a column with combined block-in and block-out time (whichever it has)")
def block_time(row):
    block_time = ''
    if row['Operation'] == 'Arrival':
        block_time = row['Actual In Block Time (Aerobahn)']
    if row['Operation'] == 'Departure':
        block_time = row['Actual Off Block Time (Aerobahn)']
    return block_time

ops_data_cleaned['block_time'] = ops_data_cleaned.apply(block_time, axis=1)
ops_data_cleaned.head(3)

Making a column with combined block-in and block-out time (whichever it has)


,Carrier Group,Call Sign,Registration,Model,Operation,Origination Airport,Destination Airport,Gate Assigned (Aerobahn),Gate,International or Domestic Indicator,Event Time,Scheduled Off Block Time (Aerobahn),Actual Off Block Time (Aerobahn),Movement Area Entrance Time,Actual Take Off Time (Aerobahn),Actual Landing Time (Aerobahn),Scheduled In Block Time (Aerobahn),Actual In Block Time (Aerobahn),Total Taxi Time,block_time
1,United Airlines,UAL385,N57439,37K,Arrival,IAH,SFO,Gate_F73,Gate_F73,Domestic,9/12/19 0:02,9/11/19 16:31,9/11/19 20:10,NaN,9/11/19 20:27,9/12/19 0:02,9/11/19 20:44,9/12/19 0:06,0:04:25,9/12/19 0:06
2,Alaska Airlines,SKW3395,N404SY,E75,Arrival,PSP,SFO,HardStand_2_A13_A13B,HardStand_2_A13_A13B,Domestic,9/12/19 0:04,9/11/19 20:40,9/11/19 22:32,NaN,9/11/19 22:57,9/12/19 0:04,9/11/19 22:10,9/12/19 0:13,0:09:16,9/12/19 0:13
3,Delta Air Lines,DAL2416,N115DU,BCS1,Arrival,SLC,SFO,Gate_C42,Gate_C42,Domestic,9/12/19 0:05,9/11/19 21:25,9/11/19 22:00,NaN,9/11/19 22:31,9/12/19 0:05,9/11/19 23:30,9/12/19 0:08,0:03:25,9/12/19 0:08


In [6]:
# Looks good, let's just drop any empty block_times
print("Contains %d values now" % ops_data_cleaned.shape[0])
print("Deleting rows with no new block_time data...")
ops_data_cleaned = ops_data_cleaned.dropna(subset=['block_time'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

Contains 149255 values now
Deleting rows with no new block_time data...
Contains 149190 values now


In [7]:
# Let's do the same thing we did for block_time, for gate. We've got two gate columns, let's 
# take whichever is more accurate or non-empty (Gate > Gate Assigned (Aerobahn) per our definition)
print("Combining gate columns")
def choose_gate(row):
    gate = ''
    has_aerobahn = row['Gate Assigned (Aerobahn)'] and (row['Gate Assigned (Aerobahn)'] == row['Gate Assigned (Aerobahn)']) and len(row['Gate Assigned (Aerobahn)']) > 0 and row['Gate Assigned (Aerobahn)'] != 'NaN'
    has_gate = row['Gate'] and (row['Gate'] == row['Gate']) and len(row['Gate']) > 0 and row['Gate'] != 'NaN'
    if has_aerobahn and has_gate:
        return row['Gate']
    if has_aerobahn:
        return row['Gate Assigned (Aerobahn)']
    if has_gate:
        return row['Gate']
    return None
ops_data_cleaned['calculated_gate'] = ops_data_cleaned.apply(choose_gate, axis=1)

print("\nDeleting rows with no gate data...")
ops_data_cleaned = ops_data_cleaned.dropna(subset=['calculated_gate'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

Combining gate columns

Deleting rows with no gate data...
Contains 149190 values now


In [8]:
# Let's sort our dataframe by gate and make sure things look good
ops_data_sorted = ops_data_cleaned.sort_values(['calculated_gate', 'block_time'], ascending=True)
ops_data_sorted.head(10)

,Carrier Group,Call Sign,Registration,Model,Operation,Origination Airport,Destination Airport,Gate Assigned (Aerobahn),Gate,International or Domestic Indicator,...,Scheduled Off Block Time (Aerobahn),Actual Off Block Time (Aerobahn),Movement Area Entrance Time,Actual Take Off Time (Aerobahn),Actual Landing Time (Aerobahn),Scheduled In Block Time (Aerobahn),Actual In Block Time (Aerobahn),Total Taxi Time,block_time,calculated_gate
105538,United Airlines,UAL863,N27964,78Z,Departure,SFO,SYD,*102,NaN,International,...,10/15/19 22:55,10/15/19 22:57,NaN,10/15/19 23:15,10/16/19 13:28,10/16/19 13:55,10/16/19 13:38,NaN,10/15/19 22:57,*102
168435,Alaska Airlines,ASA1390,N621VA,A320,Departure,SFO,MSY,*122,NaN,Domestic,...,1/29/20 12:05,1/29/20 11:54,1/29/20 12:03,1/29/20 12:22,1/29/20 16:12,1/29/20 16:20,1/29/20 16:17,NaN,1/29/20 11:54,*122
84680,Delta Air Lines,DAL2490,N115DU,BCS1,Arrival,SEA,SFO,*14,NaN,Domestic,...,1/31/20 7:24,1/31/20 7:58,NaN,1/31/20 8:25,1/31/20 10:26,1/31/20 10:50,1/31/20 10:45,NaN,1/31/20 10:45,*14
16763,Alaska Airlines,SKW3327,N191SY,E75,Arrival,PSP,SFO,*21A,NaN,Domestic,...,10/10/19 6:10,10/10/19 6:03,NaN,10/10/19 6:21,10/10/19 7:33,10/10/19 7:40,10/10/19 7:41,0:05:59,10/10/19 7:41,*21A
101956,Alaska Airlines,SKW3404,N191SY,E75,Departure,SFO,DAL,*21A,NaN,Domestic,...,10/10/19 8:55,10/10/19 9:50,NaN,10/10/19 10:06,10/10/19 13:03,10/10/19 12:15,10/10/19 13:11,NaN,10/10/19 9:50,*21A
102487,Alaska Airlines,SKW3370,N183SY,E75,Departure,SFO,SNA,*21A,NaN,Domestic,...,10/11/19 7:00,10/11/19 6:58,NaN,10/11/19 7:11,10/11/19 8:14,10/11/19 8:40,10/11/19 8:17,NaN,10/11/19 6:58,*21A
17411,Alaska Airlines,SKW3327,N171SY,E75,Arrival,PSP,SFO,*21A,NaN,Domestic,...,10/11/19 6:10,10/11/19 5:55,NaN,10/11/19 6:13,10/11/19 7:19,10/11/19 7:40,10/11/19 7:25,0:04:19,10/11/19 7:25,*21A
102565,Alaska Airlines,SKW3404,N171SY,E75,Departure,SFO,DAL,*21A,NaN,Domestic,...,10/11/19 8:55,10/11/19 8:56,NaN,10/11/19 9:18,10/11/19 12:16,10/11/19 12:15,10/11/19 12:20,NaN,10/11/19 8:56,*21A
103128,Alaska Airlines,SKW3370,N178SY,E75,Departure,SFO,SNA,*21A,NaN,Domestic,...,10/12/19 7:00,10/12/19 7:03,NaN,10/12/19 7:16,10/12/19 8:23,10/12/19 8:40,10/12/19 8:28,NaN,10/12/19 7:03,*21A
103194,Alaska Airlines,SKW3404,N173SY,E75,Departure,SFO,DAL,*21A,NaN,Domestic,...,10/12/19 8:55,10/12/19 8:49,NaN,10/12/19 9:04,10/12/19 12:07,10/12/19 12:15,10/12/19 12:10,NaN,10/12/19 8:49,*21A


In [9]:
# Convert block time to DateTime format, and delete any rows that don't have valid DateTime values
ops_data_sorted['block_time_dt'] = pd.to_datetime(ops_data_sorted['block_time'], infer_datetime_format=True)
print("Contains %d values now" % ops_data_cleaned.shape[0])
print("Deleting rows with no valid DateTime block data...")
ops_data_cleaned = ops_data_sorted.dropna(subset=['block_time_dt'], how='all')
print("Contains %d values now" % ops_data_cleaned.shape[0])

Contains 149190 values now
Deleting rows with no valid DateTime block data...
Contains 149190 values now


In [11]:
if input_3b_location and len(input_3b_location) > 0:
    print("Changing new to old gate numbering scheme...")    
    start = dt.datetime(day=16, month=10, year=2019, hour=0, minute=0, second=0)
    mapping_with_gate_changes = pd.read_csv(input_3b_location)

    def change_new_to_old_gate_scheme(row):
        curr_gate = row['calculated_gate']
        old_gate = curr_gate
        conversion = mapping_with_gate_changes[mapping_with_gate_changes['new_gate'] == curr_gate]
        if len(conversion) > 0 and row['block_time_dt'] >= start:
            gate_change = conversion.iloc[0]['gate'] 
            return gate_change
        return old_gate
    
    ops_data_cleaned['calculated_gate'] = ops_data_cleaned.apply(change_new_to_old_gate_scheme, axis=1)
    print("New to old gate numbering scheme change completed.")

In [12]:
# Great, now we need to combine the two rows that define an arrival and departure from the gate into one row
# that will contain both. Since the dataframe is sorted, we can pretty much just go through it directly
# and grab any valid arrival that has a departure immediately following it. If it's not immediately after it,
# it's not a valid operation and shouldn't be used in our analysis.

# We'll put our result in a data frame called "Flights"
# NOTE: This operation takes some time, but TQDM will display a progress bar.

temp = []
skip = 0
curr = None
has_curr = False

columns_from_arrival = ['calculated_gate', 'Carrier Group', 'Registration', 'Model',
                        'International or Domestic Indicator', 'Actual Landing Time (Aerobahn)',
                        'Scheduled In Block Time (Aerobahn)', 'Origination Airport']

columns_from_departure = ['Destination Airport', 'Scheduled Off Block Time (Aerobahn)', ]
                        
flights = []

def addFlight(arrival, departure):
    flight = {}
    for column in columns_from_arrival:
        flight[column] = arrival[column]
    for column in columns_from_departure:
        flight[column] = departure[column]
    flight['total_taxi_time_from_arrival'] = arrival['Total Taxi Time']
    flight['total_taxi_time_from_departure'] = departure['Total Taxi Time']
    flight['call_sign_arrival'] = arrival['Call Sign']
    flight['call_sign_departure'] = departure['Call Sign']
    flight['in_block_time_dt'] = arrival['block_time_dt']
    flight['off_block_time_dt'] = departure['block_time_dt']
    flights.append(flight)
    
for index, row in tqdm(ops_data_sorted.iterrows()):
    if row['Operation'] == 'Arrival':
        curr = row
        has_curr = True
    if row['Operation'] == 'Departure':
        if has_curr and (row['Registration'] == curr['Registration']) and (row['calculated_gate'] == curr['calculated_gate']): # a match!
            if (pd.Timedelta(seconds=0) < (row['block_time_dt'] - curr['block_time_dt']) < MAX_BLOCK_DIFFERENCE_TIME):
                addFlight(arrival = curr, departure = row)
        curr = None
        has_curr = False

print("%d number of flights in our data" % len(flights))
flights = pd.DataFrame(flights)
flights.head(5)


45988 number of flights in our data


,calculated_gate,Carrier Group,Registration,Model,International or Domestic Indicator,Actual Landing Time (Aerobahn),Scheduled In Block Time (Aerobahn),Origination Airport,Destination Airport,Scheduled Off Block Time (Aerobahn),total_taxi_time_from_arrival,total_taxi_time_from_departure,call_sign_arrival,call_sign_departure,in_block_time_dt,off_block_time_dt
0,*21A,Alaska Airlines,N191SY,E75,Domestic,10/10/19 7:33,10/10/19 7:40,PSP,DAL,10/10/19 8:55,0:05:59,NaN,SKW3327,SKW3404,2019-10-10 07:41:00,2019-10-10 09:50:00
1,*21A,Alaska Airlines,N171SY,E75,Domestic,10/11/19 7:19,10/11/19 7:40,PSP,DAL,10/11/19 8:55,0:04:19,NaN,SKW3327,SKW3404,2019-10-11 07:25:00,2019-10-11 08:56:00
2,*22A,Alaska Airlines,N193SY,E75,Domestic,10/18/19 21:57,10/18/19 22:10,PSP,SNA,10/19/19 7:00,0:08:08,NaN,SKW3395,SKW3370,2019-10-18 22:14:00,2019-10-19 06:58:00
3,*22A,Alaska Airlines,N405SY,E75,Domestic,10/19/19 7:30,10/19/19 7:40,PSP,DAL,10/19/19 8:55,0:08:26,NaN,SKW3327,SKW3404,2019-10-19 07:40:00,2019-10-19 08:56:00
4,*22A,Alaska Airlines,N195SY,E75,Domestic,11/7/19 22:48,11/7/19 21:50,PSP,PDX,11/8/19 9:00,0:08:08,NaN,SKW3383,SKW3325,2019-11-07 23:04:00,2019-11-08 08:59:00


In [13]:
# Let's add a TimeDelta field defining the length of the aircraft's time sitting in the gate
# and export to CSV.

flights['block_time_delta'] = (flights['off_block_time_dt'] - flights['in_block_time_dt']).dt.seconds / 60.0
export_csv = flights.to_csv(output_3c_location, index = None, header=True)
print("Complete!")
flights.head()

Complete!


,calculated_gate,Carrier Group,Registration,Model,International or Domestic Indicator,Actual Landing Time (Aerobahn),Scheduled In Block Time (Aerobahn),Origination Airport,Destination Airport,Scheduled Off Block Time (Aerobahn),total_taxi_time_from_arrival,total_taxi_time_from_departure,call_sign_arrival,call_sign_departure,in_block_time_dt,off_block_time_dt,block_time_delta
0,*21A,Alaska Airlines,N191SY,E75,Domestic,10/10/19 7:33,10/10/19 7:40,PSP,DAL,10/10/19 8:55,0:05:59,NaN,SKW3327,SKW3404,2019-10-10 07:41:00,2019-10-10 09:50:00,129.0
1,*21A,Alaska Airlines,N171SY,E75,Domestic,10/11/19 7:19,10/11/19 7:40,PSP,DAL,10/11/19 8:55,0:04:19,NaN,SKW3327,SKW3404,2019-10-11 07:25:00,2019-10-11 08:56:00,91.0
2,*22A,Alaska Airlines,N193SY,E75,Domestic,10/18/19 21:57,10/18/19 22:10,PSP,SNA,10/19/19 7:00,0:08:08,NaN,SKW3395,SKW3370,2019-10-18 22:14:00,2019-10-19 06:58:00,524.0
3,*22A,Alaska Airlines,N405SY,E75,Domestic,10/19/19 7:30,10/19/19 7:40,PSP,DAL,10/19/19 8:55,0:08:26,NaN,SKW3327,SKW3404,2019-10-19 07:40:00,2019-10-19 08:56:00,76.0
4,*22A,Alaska Airlines,N195SY,E75,Domestic,11/7/19 22:48,11/7/19 21:50,PSP,PDX,11/8/19 9:00,0:08:08,NaN,SKW3383,SKW3325,2019-11-07 23:04:00,2019-11-08 08:59:00,595.0
